In [1]:
import pandas as pd
import json
import os

In [2]:
path = "D:\\phonepe_project\\pulse\\data\\top\\transaction\\country\\india\\state"
path_a = os.listdir(path)

In [6]:
std5 = {'state':[], 'year':[],'quater':[],'pincode':[], 'transaction_count':[], 'transaction_amount':[]}

for i in path_a:
    path_yr = path+"\\"+i+"\\"
    year = os.listdir(path_yr)

    for j in year:
        path_file = path_yr + j
        path_n = os.listdir(path_file)

        for k in path_n:
            asd = path_file + "\\" + k
            data_n = open(asd, "r")
            output = json.load(data_n)

            for l in output['data']['pincodes']:
                pincode = l['entityName']
                count = l['metric']['count']
                amount = l['metric']['amount']
                std5['pincode'].append(pincode)
                std5['transaction_count'].append(count)
                std5['transaction_amount'].append(amount)
                std5['state'].append(i)
                std5['year'].append(j)
                std5['quater'].append(int(k.strip('.json')))

df5 = pd.DataFrame(std5)


In [7]:
df5

,state,year,quater,pincode,transaction_count,transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
8205,west-bengal,2023,3,700001,3536928,7.193522e+09
8206,west-bengal,2023,3,721301,3464729,4.871566e+09
8207,west-bengal,2023,3,700039,3405551,5.095825e+09
8208,west-bengal,2023,3,732125,3347165,5.353831e+09


In [8]:
df5.to_csv(r'D:\phonepe_project\top_trans.csv')

In [9]:
import psycopg2

mydb = psycopg2.connect(
    host = "localhost",
    database = "postgres",
    user = "postgres",
    password = "123456")

mycursor = mydb.cursor()

In [10]:
query5 = """
            CREATE TABLE IF NOT EXISTS top_trans(
                state VARCHAR(255),
                year VARCHAR(4),
                quater INT, 
                pincode VARCHAR(255), 
                transaction_count INT, 
                transaction_amount NUMERIC
                )

            """ 

try:
    mycursor.execute(query5)
    mydb.commit()
    print("Tables created successfully.")
except Exception as e:
    print("Error:", e)
    mydb.rollback()

Tables created successfully.


In [11]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL database connection parameters
db_config = {
    'user': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')

# CSV file path
csv_file_path = 'D:\\phonepe_project\\top_trans.csv'

# Read CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Insert data into PostgreSQL table
try:
    df.to_sql('top_trans', con=engine, if_exists='replace', index=True)
    print("Data inserted successfully.")
except Exception as e:
    print("Error:", e)
finally:
    engine.dispose()  # Close the database connection


Data inserted successfully.
